# Running EnergyPlus from Eppy

It would be great if we could run EnergyPlus directly from our IDF wouldn’t it?

Well here’s how we can.

In [1]:
# you would normaly install eppy by doing
# python setup.py install
# or
# pip install eppy
# or
# easy_install eppy

# if you have not done so, uncomment the following three lines
import sys
# pathnameto_eppy = 'c:/eppy'
pathnameto_eppy = '../'
sys.path.append(pathnameto_eppy)

In [2]:
from eppy.modeleditor import IDF

iddfile = "/Applications/EnergyPlus-8-3-0/Energy+.idd"
IDF.setiddname(iddfile)


In [3]:
idfname = "/Applications/EnergyPlus-8-3-0/ExampleFiles/BasicsFiles/Exercise1A.idf"
epwfile = "/Applications/EnergyPlus-8-3-0/WeatherData/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw"

idf = IDF(idfname, epwfile)
idf.run()


/Applications/EnergyPlus-8-3-0/energyplus --weather /Applications/EnergyPlus-8-3-0/WeatherData/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw --output-directory /Users/santoshphilip/Documents/coolshadow/github/eppy/docs --idd /Applications/EnergyPlus-8-3-0/Energy+.idd /Users/santoshphilip/Documents/coolshadow/github/eppy/docs/in.idf




if you are in a terminal, you will see something like this::


    
    Processing Data Dictionary
    Processing Input File
    Initializing Simulation
    Reporting Surfaces
    Beginning Primary Simulation
    Initializing New Environment Parameters
    Warming up {1}
    Warming up {2}
    Warming up {3}
    Warming up {4}
    Warming up {5}
    Warming up {6}
    Starting Simulation at 07/21 for CHICAGO_IL_USA COOLING .4% CONDITIONS DB=>MWB
    Initializing New Environment Parameters
    Warming up {1}
    Warming up {2}
    Warming up {3}
    Warming up {4}
    Warming up {5}
    Warming up {6}
    Starting Simulation at 01/21 for CHICAGO_IL_USA HEATING 99.6% CONDITIONS
    Writing final SQL reports
    EnergyPlus Run Time=00hr 00min  0.24sec


It’s as simple as that to run using the EnergyPlus defaults, but all the EnergyPlus command line interface options are also supported.

To get a description of the options available, as well as the defaults you can call the Python built-in help function on the IDF.run method and it will print a full description of the options to the console.

In [4]:
help(idf.run)

Help on method run in module eppy.modeleditor:

run(**kwargs) method of eppy.modeleditor.IDF instance
    This method wraps the following method:
    
    run(idf=None, weather=None, output_directory='', annual=False, design_day=False, idd=None, epmacro=False, expandobjects=False, readvars=False, output_prefix=None, output_suffix=None, version=False, verbose='v', ep_version=None)
        Wrapper around the EnergyPlus command line interface.
        
        Parameters
        ----------
        idf : str
            Full or relative path to the IDF file to be run, or an IDF object.
        
        weather : str
            Full or relative path to the weather file.
        
        output_directory : str, optional
            Full or relative path to an output directory (default: 'run_outputs)
        
        annual : bool, optional
            If True then force annual simulation (default: False)
        
        design_day : bool, optional
            Force design-day-only simulati

*Note:* idf.run() works for E+ version >= 8.3

## Running in parallel processes

If you have acomputer with multiple cores, you may want to use all the cores. EnergyPlus allows you to run simulations on multiple cores. 

Here is an example script of how use ``eppy`` to run on multiple cores

In [ ]:
"""multiprocessing runs"""


import os 
from eppy.modeleditor import IDF
from eppy.runner.run_functions import runIDFs

def make_eplaunch_options(idf):
    """Make options for run, so that it runs like EPLaunch on Windows"""
    idfversion = idf.idfobjects['version'][0].Version_Identifier.split('.')
    idfversion.extend([0] * (3 - len(idfversion)))
    idfversionstr = '-'.join([str(item) for item in idfversion])
    fname = idf.idfname
    options = {
        'ep_version':idfversionstr, # runIDFs needs the version number
        'output_prefix':os.path.basename(fname).split('.')[0],
        'output_suffix':'C',
        'output_directory':os.path.dirname(fname),
        'readvars':True,
        'expandobjects':True
        }
    return options




def main():
    iddfile = "/Applications/EnergyPlus-9-3-0/Energy+.idd" # change this for your operating system
    IDF.setiddname(iddfile)
    epwfile = "USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"


    runs = []

    # File is from the Examples Folder
    idfname = "HVACTemplate-5ZoneBaseboardHeat.idf"
    idf = IDF(idfname, epwfile)
    theoptions = make_eplaunch_options(idf) 
    runs.append([idf, theoptions])

    # copy of previous file
    idfname = "HVACTemplate-5ZoneBaseboardHeat1.idf"
    idf = IDF(idfname, epwfile)
    theoptions = make_eplaunch_options(idf)
    runs.append([idf, theoptions])

    num_CPUs = 2
    runIDFs(runs, num_CPUs)

if __name__ == '__main__':
    main()

## Running in parallel processes using Generators

Maybe you want to run a 100 or a 1000 simulations. The code above will not let you do that, since it will try to load 1000 files into memory. 

Now you need to use generators (python's secret sauce. if you don't know this, you need to look into it). 

Here is a code using generators. Now you can simulate a 1000 files

### gmulti.py

In [ ]:
"""multiprocessing runs

using generators instead of a list
when you are running a 100 files you have to use generators"""

import os 
from eppy.modeleditor import IDF
from eppy.runner.run_functions import runIDFs

def make_eplaunch_options(idf):
    """Make options for run, so that it runs like EPLaunch on Windows"""
    idfversion = idf.idfobjects['version'][0].Version_Identifier.split('.')
    idfversion.extend([0] * (3 - len(idfversion)))
    idfversionstr = '-'.join([str(item) for item in idfversion])
    fname = idf.idfname
    options = {
        'ep_version':idfversionstr, # runIDFs needs the version number
        'output_prefix':os.path.basename(fname).split('.')[0],
        'output_suffix':'C',
        'output_directory':os.path.dirname(fname),
        'readvars':True,
        'expandobjects':True
        }
    return options




def main():
    iddfile = "/Applications/EnergyPlus-9-3-0/Energy+.idd"
    IDF.setiddname(iddfile)
    epwfile = "USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"



    # File is from the Examples Folder
    idfname1 = "HVACTemplate-5ZoneBaseboardHeat.idf"
    # copy of previous file
    idfname2 = "HVACTemplate-5ZoneBaseboardHeat1.idf"


    fnames = [idfname1, idfname1]
    idfs = (IDF(fname, epwfile) for fname in fnames)
    runs = ((idf, make_eplaunch_options(idf) ) for idf in idfs)


    num_CPUs = 2
    runIDFs(runs, num_CPUs)

if __name__ == '__main__':
    main()

## True Multi-processing

What if you want to run your simulations on multiple computers. What if those computers are on other networks (some at home and the other in your office and others in your server room) and some on the cloud.

There is an experimental repository where you can do this. Keep an eye on this:

https://github.com/pyenergyplus/zeppy

## Make idf.run() work like EPLaunch

I like the function ``make_eplaunch_options``. Can I use it to do a single run ?

Yes! You can. 

**An Explanation:** EPLaunch is an application that comes with EnergyPlus on the Windows Platform. It has a default functionality that people become familiar with and come to expect. ``make_eplaunch_options`` set up the idf.run() arguments so that it behaves in the same way as EPLaunch.

Here is the Sample code below. Modify and use it for your needs

In [ ]:
"""single run EPLaunch style"""


import os 
from eppy.modeleditor import IDF
from eppy.runner.run_functions import runIDFs

def make_eplaunch_options(idf):
    """Make options for run, so that it runs like EPLaunch on Windows"""
    idfversion = idf.idfobjects['version'][0].Version_Identifier.split('.')
    idfversion.extend([0] * (3 - len(idfversion)))
    idfversionstr = '-'.join([str(item) for item in idfversion])
    fname = idf.idfname
    options = {
        # 'ep_version':idfversionstr, # runIDFs needs the version number
            # idf.run does not need the above arg
            # you can leave it there and it will be fine :-)
        'output_prefix':os.path.basename(fname).split('.')[0],
        'output_suffix':'C',
        'output_directory':os.path.dirname(fname),
        'readvars':True,
        'expandobjects':True
        }
    return options


def main():
    iddfile = "/Applications/EnergyPlus-9-3-0/Energy+.idd" # change this for your operating system and E+ version
    IDF.setiddname(iddfile)
    epwfile = "USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"



    # File is from the Examples Folder
    idfname = "HVACTemplate-5ZoneBaseboardHeat.idf"
    idf = IDF(idfname, epwfile)
    theoptions = make_eplaunch_options(idf)
    idf.run(**theoptions)


if __name__ == '__main__':
    main()

## Debugging and reporting problems

Debugging issues with IDF.run() used to be difficult, since you needed to go and hunt for the eplusout.err file, and the error message returned was not at all helpful.

Now the output from EnergyPlus is returned in the error message, as well as the location and contents of eplusout.err.

For example, this is the error message produced when running an IDF which contains an “HVACTemplate:Thermostat” object without passing expand_objects=True to idf.run():

In [ ]:
E           eppy.runner.run_functions.EnergyPlusRunError:
E           Program terminated: EnergyPlus Terminated--Error(s) Detected.
E
E           Contents of EnergyPlus error file at C:\Users\jamiebull1\git\eppy\eppy\tests\test_dir\eplusout.err
E           Program Version,EnergyPlus, Version 8.9.0-40101eaafd, YMD=2018.10.14 20:49,
E              ** Severe  ** Line: 107 You must run the ExpandObjects program for "HVACTemplate:Thermostat"
E              **  Fatal  ** Errors occurred on processing input file. Preceding condition(s) cause termination.
E              ...Summary of Errors that led to program termination:
E              ..... Reference severe error count=1
E              ..... Last severe error=Line: 107 You must run the ExpandObjects program for "HVACTemplate:Thermostat"
E              ************* Warning:  Node connection errors not checked - most system input has not been read (see previous warning).
E              ************* Fatal error -- final processing.  Program exited before simulations began.  See previous error messages.
E              ************* EnergyPlus Warmup Error Summary. During Warmup: 0 Warning; 0 Severe Errors.
E              ************* EnergyPlus Sizing Error Summary. During Sizing: 0 Warning; 0 Severe Errors.
E              ************* EnergyPlus Terminated--Fatal Error Detected. 0 Warning; 1 Severe Errors; Elapsed Time=00hr 00min  0.16sec

When reporting any issues related to IDF.run(), please include the error message as well as the code used to call the function. Also, it may help to include an IDF and EPW which display the problem so that we can try to reproduce it.